# Website Scrapper

# Import all packages

In [34]:
import re
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import csv

# Scrap a Single Page
- Using beautiful soup.
- Exract the following properties
    - heading
    - pricing
    - numberOfBedrooms = container.find(itemprop="numberOfBedrooms")
    - numberOfBathrooms
    - floorSize = container.find(itemprop="floorSize")

In [35]:
def scrape_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        containers = soup.find_all(class_=re.compile(r'ListItemCard_container__[a-zA-Z0-9]+'))
        all_buildings = []
        
        for container in containers:
            # Within each container, find the heading with the specified class pattern
            heading = container.find(class_=re.compile(r'CellHeading_heading__[a-zA-Z0-9]+'))
            pricing = container.find(class_=re.compile(r'CellPrice_text__[a-zA-Z0-9]+'))
            numberOfBedrooms = container.find(itemprop="numberOfBedrooms")
            numberOfBathroomsTotal = container.find(itemprop="numberOfBathroomsTotal")
            floorSize = container.find(itemprop="floorSize")
            
            if heading:
                templist = []
                templist.append(heading.text.strip())
                templist.append(pricing.text.strip())
                templist.append(numberOfBedrooms.text.strip())
                templist.append(numberOfBathroomsTotal.text.strip())
                templist.append(floorSize.text.strip())
                all_buildings.append(templist)

        return all_buildings
    else:
        print("Failed to retrieve page:", response.status_code)

In [36]:
def scrape_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        containers = soup.find_all(class_=re.compile(r'ListItemCard_container__[a-zA-Z0-9]+'))
        all_buildings = []
        
        for container in containers:
            # Within each container, find the heading with the specified class pattern
            heading = container.find(class_=re.compile(r'CellHeading_heading__[a-zA-Z0-9]+'))
            pricing = container.find(class_=re.compile(r'CellPrice_text__[a-zA-Z0-9]+'))
            numberOfBedrooms = container.find(itemprop="numberOfBedrooms")
            numberOfBathroomsTotal = container.find(itemprop="numberOfBathroomsTotal")
            floorSize = container.find(itemprop="floorSize")
            
            if heading:
                templist = []
                templist.append(heading.text.strip())
                templist.append(pricing.text.strip())
                templist.append(numberOfBedrooms.text.strip())
                templist.append(numberOfBathroomsTotal.text.strip())
                templist.append(floorSize.text.strip())
                all_buildings.append(templist)


        return all_buildings
    else:
        print("Failed to retrieve page:", response.status_code)

In [37]:
def scraper_ind_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        description = soup.find(class_=re.compile(r'Description_description__[a-zA-Z0-9]+'))
        return(description.text.strip())
    else:
        print("Failed to retrieve page:", response.status_code)
        
scraper_ind_page("https://www.99.co/singapore/sale/property/159-bedok-south-avenue-3-hdb-2JEoAwCqbMuLWs2ztdH6BU")

'4S next to corner\nHigh floor\nUnblock view\nNo noon sun\nOriginal condition\nSuit for design your own dream home\nCan contra\nPurely selling\nNo extension needed\nWalking distance to future MRT station\nNear Temasek Primary and Secondary school, childcare center, CC, market and hawker center etc\nOnly Chinese and Malay buyers eligible\nAll SPRs can buy\nFor exclusive viewing, kindly contact Alan Poh @  9100 XXXX.'

# Selenium to navigate page

- Unable to find the href of the next page.
- Using Selenium to force locate the next page.

In [38]:
def get_next_page_url(url):
    # Create a WebDriver instance (assuming you have Chrome installed)
    driver = webdriver.Chrome()

    try:
        # Load the webpage
        driver.get(url)

        # Find the "Next page" button element
        next_page_button = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//a[@aria-label="Next page"]'))
        )

        # Click the "Next page" button
        next_page_button.click()

        # Retrieve the URL of the next page
        next_page_url = driver.current_url
        return next_page_url

    finally:
        # Close the WebDriver
        driver.quit()

In [39]:
def scrap_inner_page(url):
    # Set up Selenium WebDriver
    driver = webdriver.Chrome()  # or any other WebDriver you're using
    driver.get(url)

    try:
        # Find all elements with data-cy="listingCard" to navigate to the next pages
        next_page_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-cy="listingCard"]')))
        
        # Iterate through each element and click it to navigate to the respective page
        for next_page_element in next_page_elements:
            next_page_element.click()

            # Wait for the next page to load
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-cy="listingCard"]')))

            # Switch to the new tab
            driver.switch_to.window(driver.window_handles[-1])

            # Perform scraping on the current page
            scraper_ind_page(driver.current_url)
            
            # Close the current tab to return to the previous page for clicking the next element
            driver.close()

            # Switch back to the main tab
            driver.switch_to.window(driver.window_handles[0])
    finally:
        # Close the WebDriver session
        driver.quit()

# URL of the initial page
initial_url = "https://www.99.co/singapore/sale?main_category=hdb"

# Call the function to perform scraping on the next page
scrap_inner_page(initial_url)


Failed to retrieve page: 429


WebDriverException: Message: no such execution context
  (Session info: chrome=123.0.6312.58)
Stacktrace:
0   chromedriver                        0x00000001027f43e8 chromedriver + 4326376
1   chromedriver                        0x00000001027ec8b0 chromedriver + 4294832
2   chromedriver                        0x0000000102417eec chromedriver + 278252
3   chromedriver                        0x0000000102401748 chromedriver + 186184
4   chromedriver                        0x00000001023ffe9c chromedriver + 179868
5   chromedriver                        0x0000000102400930 chromedriver + 182576
6   chromedriver                        0x000000010240f3b0 chromedriver + 242608
7   chromedriver                        0x0000000102422504 chromedriver + 320772
8   chromedriver                        0x0000000102400edc chromedriver + 184028
9   chromedriver                        0x0000000102422188 chromedriver + 319880
10  chromedriver                        0x000000010249343c chromedriver + 783420
1

# Search a few pages automatically

In [5]:
def get_pages(url, number, result=[]):
    if number <= 0:
        return result
    
    result += scrape_page(url)
    next_page_url = get_next_page_url(url)
    
    if next_page_url:
        return get_pages(next_page_url, number - 1, result)
    else:
        return result
    
starting_url = "https://www.99.co/singapore/sale?main_category=hdb"
number_of_pages_to_scrape = 2
result = get_pages(starting_url, number_of_pages_to_scrape)

for res in result:
    print(res)

['3 Room HDB in 308C Punggol Walk', '$800,000', '3 Beds', '2 Baths', '1,206 sqft / 112.04 sqm']
['Exec HDB in 136 Pasir Ris Street 11', '$810,000', '4 Beds', '2 Baths', '1,560 sqft / 144.93 sqm']
['5 Room HDB in 293 Yishun Street 22', '$750,000', '4 Beds', '2 Baths', '1,431 sqft / 132.94 sqm']
['3 Room HDB in 462D Yishun Avenue 6', '$465,000', '2 Beds', '2 Baths', '732 sqft / 68 sqm']
['3 Room HDB in 422 Bukit Batok West Avenue 2', '$380,000', '2 Beds', '2 Baths', '731 sqft / 67.91 sqm']
['Exec HDB in 150 Mei Ling Street', '$1,300,000', '4 Beds', '3 Baths', '1,582 sqft / 146.97 sqm']
["4 Room HDB in 26B Saint George's Lane", '$970,000', '3 Beds', '2 Baths', '990 sqft / 91.97 sqm']
['Exec HDB in 637 Woodlands Ring Road', '$1,150,000', '3 Beds', '3 Baths', '1,528 sqft / 141.96 sqm']
['4 Room HDB in 11 Haig Road', '$680,000', '2 Beds', '2 Baths', '990 sqft / 91.97 sqm']
['4 Room HDB in 817B Keat Hong Link', '$588,000', '3 Beds', '2 Baths', '990 sqft / 91.97 sqm']
['4 Room HDB in 142 Loron

# Save to CSV

In [6]:
def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Write header row
        writer.writerow(['Title', 'Price', 'Bedrooms', 'Bathrooms', 'Floor Size'])
        # Write data rows
        writer.writerows(data)

starting_url = "https://www.99.co/singapore/sale?main_category=hdb"
number_of_pages_to_scrape = 2
result = get_pages(starting_url, number_of_pages_to_scrape)

# Specify the filename for the CSV file
csv_filename = "hdb_listings.csv"

# Save the scraped data into the CSV file
save_to_csv(result, csv_filename)